# Imports

In [1]:
'''Train a simple deep CNN on the CIFAR10 small images dataset.

It gets to 75% validation accuracy in 25 epochs, and 79% after 50 epochs.
(it's still underfitting at that point, though).
'''

from __future__ import print_function

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import csv
import keras
import numpy as np
from keras import regularizers
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping
from itertools import chain
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

def limit_mem():
    keras.backend.get_session().close()
    cfg = keras.backend.tf.ConfigProto()
    cfg.gpu_options.allow_growth = True
    keras.backend.set_session(keras.backend.tf.Session(config=cfg))

get_available_gpus()

['/device:GPU:0']

In [10]:
#Choose layers to get nulled!

def generate_layer_mask(active_layers, layers_num):
    layer_mask = [0] * layers_num
    for layer in active_layers:
        layer_mask[layer] = 1
    return layer_mask

def show_weights_distribution(cnn_model, xrange=0.4):
    #NO BIAS WEIGHTS
    all_weights = []
    for x in cnn_model.layers:
        if(x.get_weights()) :
            all_weights.append(np.reshape(x.get_weights()[0], -1))
    all_weights = np.concatenate(all_weights)
    plt.hist(all_weights, bins=80, range=(-xrange, xrange))

def get_threshold(cnn_model, layer_mask):
    #NO BIAS WEIGHTS
    all_weights = []
    layer_mask_index = 0
    
    for x in cnn_model.layers:
        if(x.get_weights() and layer_mask[layer_mask_index] == 1) :
            all_weights.append(np.reshape(x.get_weights()[0], -1))
        layer_mask_index += 1
            
    all_weights = np.abs(np.concatenate(all_weights))
    all_weights = np.sort(all_weights)
    threshold_value = all_weights[int(zero_part * len(all_weights)) + 1]
    return threshold_value
    
def get_masks(cnn_model, threshold_value, layer_mask):
    #NO BIAS WEIGHTS
    zero_masks = []
    layer_mask_index = 0
    
    for x in cnn_model.layers:
        if(x.get_weights() and layer_mask[layer_mask_index] == 1) :
            zero_masks.append(np.abs(x.get_weights()[0]) <= threshold_value)
        layer_mask_index += 1
            
    return zero_masks

def set_null_weights_by_mask(cnn_model, zero_mask, layer_mask):
    #NO BIAS WEIGHTS
    mask_iter = 0
    layer_mask_index = 0
    
    for x in cnn_model.layers:
        if(x.get_weights() and layer_mask[layer_mask_index] == 1) :
            old_weights, old_bias = x.get_weights()
            old_weights[zero_mask[mask_iter]] = 0
            mask_iter += 1
            x.set_weights([old_weights, old_bias])
        layer_mask_index += 1

# Model achitecture

In [11]:
batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name_prefix = 'keras_cf10_'
l1_coef = 0.0001 # * 10, * 100, * ??? was 0.00001
zero_part = 0.0
zero_step = 0.05
min_los_delta = 0.001

# 01 version - stock model, dropouts, no L1/L2
# 02 version - stock model, no dropouts, no L1/L2
# 03 version - stock model, no dropouts, L1 0.01 kernel_regularizer

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', 
                 input_shape=x_train.shape[1:]))#, kernel_regularizer=regularizers.l1(l1_coef))) # Layer 0
model.add(Activation('relu')) # Layer 1
model.add(Conv2D(32, (3, 3)))#, kernel_regularizer=regularizers.l1(l1_coef))) # Layer 2
model.add(Activation('relu')) # Layer 3
model.add(MaxPooling2D(pool_size=(2, 2))) # Layer 4
model.add(Dropout(0.25)) # Layer 5

model.add(Conv2D(64, (3, 3), padding='same'))#, kernel_regularizer=regularizers.l1(l1_coef))) # Layer 6
model.add(Activation('relu')) # Layer 7
model.add(Conv2D(64, (3, 3)))#, kernel_regularizer=regularizers.l1(l1_coef))) # Layer 8
model.add(Activation('relu')) # Layer 9
model.add(MaxPooling2D(pool_size=(2, 2))) # Layer 10
model.add(Dropout(0.25)) # Layer 11

model.add(Flatten()) # Layer 12
model.add(Dense(512))#, kernel_regularizer=regularizers.l1(l1_coef)))  # Layer 13
model.add(Activation('relu'))  # Layer 14
model.add(Dropout(0.5))  # Layer 15 
model.add(Dense(num_classes))#, kernel_regularizer=regularizers.l1(l1_coef)))  # Layer 16
model.add(Activation('softmax'))  # Layer 17

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
# maybe adam, /5 or /10 default parametrs

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, "empty_model")
model.save(model_path)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


# Callbacks

In [12]:
cb_lists = [
    EarlyStopping(monitor='val_loss', 
                  min_delta=0.1 * min_los_delta, 
                  patience=10, 
                  verbose=0, 
                  mode='auto', 
                  baseline=None, 
                  restore_best_weights=False)
]

# Example Loop

In [13]:
def example_loop():
    if not data_augmentation:
        print('Not using data augmentation.')
        model.fit(x_train, y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_data=(x_test, y_test),
                  shuffle=True)
    else:
        print('Using real-time data augmentation.')
        # This will do preprocessing and realtime data augmentation:
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            zca_epsilon=1e-06,  # epsilon for ZCA whitening
            rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
            # randomly shift images horizontally (fraction of total width)
            width_shift_range=0.1,
            # randomly shift images vertically (fraction of total height)
            height_shift_range=0.1,
            shear_range=0.,  # set range for random shear
            zoom_range=0.,  # set range for random zoom
            channel_shift_range=0.,  # set range for random channel shifts
            # set mode for filling points outside the input boundaries
            fill_mode='nearest',
            cval=0.,  # value used for fill_mode = "constant"
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False,  # randomly flip images
            # set rescaling factor (applied before any other transformation)
            rescale=None,
            # set function that will be applied on each input
            preprocessing_function=None,
            # image data format, either "channels_first" or "channels_last"
            data_format=None,
            # fraction of images reserved for validation (strictly between 0 and 1)
            validation_split=0.0)
    
        # Compute quantities required for feature-wise normalization
        # (std, mean, and principal components if ZCA whitening is applied).
        datagen.fit(x_train)
    
        # Fit the model on the batches generated by datagen.flow().
        model.fit_generator(datagen.flow(x_train, y_train,
                                         batch_size=batch_size),
                            epochs=epochs,
                            validation_data=(x_test, y_test),
                            workers=4,
                            steps_per_epoch=len(x_train)/batch_size,
                            callbacks=cb_lists)

# Custom Loop

In [18]:
def custom_loop():
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)
    
    enable_early_cut_off = False
    enable_saving_best_model = True
    prev_loss = 0
    best_loss = 999999999999
    best_acc = 0
    
    # Zeroing weights params
    enable_zeroing_weights = True
    zero_mode = False
    zero_mask = []
    non_zeroing_epochs = 10
    zeroing_epochs = 5
    left_cur_state_epochs = non_zeroing_epochs
    layers_to_zero = generate_layer_mask([0, 2, 6, 8], len(model.layers)) # only conv layers
    
    for i in range(0, epochs):
        gen = datagen.flow(x_train, y_train, batch_size=batch_size)
        
        if enable_zeroing_weights:
            left_cur_state_epochs -= 1
            #print('Decreased left_cur_state_epochs: ' + str(left_cur_state_epochs))
            if (left_cur_state_epochs < 0):
                zero_mode = not zero_mode
                left_cur_state_epochs = zeroing_epochs if zero_mode else non_zeroing_epochs
                if zero_mode:
                    print('Enable zero mode!')
                    zero_mask = get_masks(model, get_threshold(model, layers_to_zero), layers_to_zero)
                else:
                    print('Disable zero mode!')
        
        for j in range(0, int(len(x_train)/batch_size)):
            x_tr, y_tr = next(gen)
            model.train_on_batch(x_tr, y_tr)
            if zero_mode:
                #print('Zeroing!')
                set_null_weights_by_mask(model, zero_mask, layers_to_zero)
        
        print('Finish epoch: ' + str(i))
        epoch_loss, epoch_acc = model.evaluate(x_test, y_test)
        print('Validation loss: ' + str(epoch_loss) + ', acc ' + str(epoch_acc))
        
        if (enable_early_cut_off):
            if (abs(epoch_loss - prev_loss) < min_los_delta):
                print('Exit because loss stoped changing. epoch_loss ' 
                      + str(epoch_loss) 
                      + ' prev_loss' 
                      + str(prev_loss))
                break
            prev_loss = epoch_loss
                
        if(enable_saving_best_model):
            if(epoch_loss < best_loss):
                best_loss = epoch_loss
                best_acc = epoch_acc
                
                if not os.path.isdir(save_dir):
                    os.makedirs(save_dir)
                model_path = os.path.join(save_dir, model_name)
                model.save(model_path)
                print('Saved best model at %s ' % model_path)
    return best_loss, best_acc
            

# Just one loop

In [19]:
model_name = "example_arch"
loss_result, acc_result = custom_loop()
print('After runs, ' + str(loss_result) + ' loss and ' + str(acc_result) + ' acc')

Finish epoch: 0
10000/10000 [==============================] - 0s 43us/step
Validation loss: 0.9132600858688354, acc 0.6777
Saved best model at /home/nikitam/sparse-net-study/saved_models/example_arch 
Finish epoch: 1
10000/10000 [==============================] - 0s 48us/step
Validation loss: 0.8882406951904297, acc 0.689
Saved best model at /home/nikitam/sparse-net-study/saved_models/example_arch 
Finish epoch: 2
10000/10000 [==============================] - 1s 55us/step
Validation loss: 0.8896574728012085, acc 0.6918
Finish epoch: 3
10000/10000 [==============================] - 0s 47us/step
Validation loss: 0.8390262379646302, acc 0.707
Saved best model at /home/nikitam/sparse-net-study/saved_models/example_arch 
Finish epoch: 4
10000/10000 [==============================] - 0s 44us/step
Validation loss: 0.835950332069397, acc 0.7128
Saved best model at /home/nikitam/sparse-net-study/saved_models/example_arch 
Finish epoch: 5
10000/10000 [==============================] - 0s 46us/

10000/10000 [==============================] - 0s 38us/step
Validation loss: 0.6999618347167968, acc 0.7667
Finish epoch: 55
10000/10000 [==============================] - 0s 45us/step
Validation loss: 0.692422728729248, acc 0.7661
Finish epoch: 56
10000/10000 [==============================] - 0s 39us/step
Validation loss: 0.7515422726631165, acc 0.7513
Finish epoch: 57
10000/10000 [==============================] - 0s 40us/step
Validation loss: 0.6773226615905762, acc 0.7756
Finish epoch: 58
10000/10000 [==============================] - 0s 46us/step
Validation loss: 0.6771267597675323, acc 0.7767
Finish epoch: 59
10000/10000 [==============================] - 0s 41us/step
Validation loss: 0.7522808364868164, acc 0.7579
Finish epoch: 60
10000/10000 [==============================] - 0s 42us/step
Validation loss: 0.705610725069046, acc 0.7622
Enable zero mode!
Finish epoch: 61
10000/10000 [==============================] - 0s 40us/step
Validation loss: 0.7622931790351868, acc 0.7454
F

# Active cell with different zero part

In [ ]:
while(zero_part < 1.0):
    model = keras.models.load_model('saved_models/empty_model')
    model_name = model_name_prefix + str(zero_part) + 'zrpt'
    loss_result, acc_result = custom_loop()
    print('After runs, ' + str(loss_result) + ' loss and ' + str(acc_result) + ' acc')

    # show_weights_distribution()

    with open('results.csv', 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([str(zero_part)] + [str(loss_result)] + [str(acc_result)])
    zero_part += zero_step
        
#limit_mem()

# Load best model
#model = keras.models.load_model('saved_models/keras_cifar10_trained_03.h5')
#scores = model.evaluate(x_test, y_test, verbose=1)
#print('Test loss:', scores[0])
#print('Test accuracy:', scores[1])

# Another useless cells

In [ ]:
#!jupyter notebook stop 11440
os._exit(0)

In [ ]:
#Load model and show it's score
model = keras.models.load_model('saved_models/keras_cifar10_trained_02.h5')

scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
with open('results.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in spamreader:
        print(', '.join(row))

In [ ]:
model = keras.models.load_model('saved_models/keras_cf10_0.9500000000000003zrpt')

scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

show_weights_distribution(1)

In [ ]:
model.summary()

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

In [ ]:
!pip uninstall GraphViz